# PARTE 3 E 4: REGRESSÃO LOGÍSTICA E SVM

In [94]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# PARTE 3: Regressão Logística

1. Utilize a base de dados construída no Trabalho 3 ‘risco_credito.pkl’, que possui 14 registros, para testar o algoritmo de Regressão Logística.

2. Faça o Encoder dos dados e, para facilitar, como fizemos na aula teórica, apague os registros que possuem a classe ‘moderado’. No total teremos 11 registros.

3. Treine o algoritmo de regressão logística e utilize o parâmetro ‘random_state =1’ para ter sempre o mesmo resultado.

4. Utilize o comando ‘.intercept_’ para ter o resultado do B0.
O resultado deve ser =-0.80828993

5. Utilize o comando ‘.coef_’ para ter o resultado dos demais parâmetros que deve ser:
array([[-0.76704533,  0.23906678, -0.47976059,  1.12186218]])

6. Agora utilize o comando ‘predict’ para fazer o teste do seu algoritmo com:

    a) história boa, dívida alta, garantias nenhuma, renda > 35
    (o resultado desse teste deve ser ‘baixo’)

    b) história ruim, dívida alta, garantias adequada, renda < 15
    (o resultado desse teste deve ser ‘alto’)

In [95]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
import pickle

## 1. Carregar a base de dados risco_credito.pkl

In [96]:
dataset_risco_credito = pd.read_csv('dataset_risco_credito.csv')
print("Dataset original:")
print(dataset_risco_credito)
print(f"\nTotal de registros: {len(dataset_risco_credito)}")

Dataset original:
        historia divida garantias     renda     risco
0           ruim   alta   nenhuma      0_15      alto
1   desconhecida   alta   nenhuma     15_35      alto
2   desconhecida  baixa   nenhuma     15_35  moderado
3   desconhecida  baixa   nenhuma  acima_35      alto
4   desconhecida  baixa   nenhuma  acima_35     baixo
5   desconhecida  baixa  adequada  acima_35     baixo
6           ruim  baixa   nenhuma      0_15      alto
7           ruim  baixa  adequada  acima_35  moderado
8            boa  baixa   nenhuma  acima_35     baixo
9            boa   alta  adequada  acima_35     baixo
10           boa   alta   nenhuma      0_15      alto
11           boa   alta   nenhuma     15_35  moderado
12           boa   alta   nenhuma  acima_35     baixo
13          ruim   alta   nenhuma     15_35      alto

Total de registros: 14


## 2. Remover registros com classe 'moderado' e fazer o encoding

In [97]:
dataset_filtrado = dataset_risco_credito[dataset_risco_credito['risco'] != 'moderado'].copy()
print(f"\nDataset após remover 'moderado':")
print(dataset_filtrado)
print(f"\nTotal de registros: {len(dataset_filtrado)}")

X_risco_credito_filtrado = dataset_filtrado.iloc[:, 0:-1].values
y_risco_credito_filtrado = dataset_filtrado.iloc[:, -1].values

X_encoded = X_risco_credito_filtrado.copy()
label_encoders = []

for i in range(X_encoded.shape[1]):
    le = LabelEncoder()
    X_encoded[:, i] = le.fit_transform(X_encoded[:, i])
    label_encoders.append(le)

le_y = LabelEncoder()
y_encoded = le_y.fit_transform(y_risco_credito_filtrado)

print("\nX codificado:")
print(X_encoded)
print("\ny codificado:")
print(y_encoded)
print(f"\nClasses: {le_y.classes_}")


Dataset após remover 'moderado':
        historia divida garantias     renda  risco
0           ruim   alta   nenhuma      0_15   alto
1   desconhecida   alta   nenhuma     15_35   alto
3   desconhecida  baixa   nenhuma  acima_35   alto
4   desconhecida  baixa   nenhuma  acima_35  baixo
5   desconhecida  baixa  adequada  acima_35  baixo
6           ruim  baixa   nenhuma      0_15   alto
8            boa  baixa   nenhuma  acima_35  baixo
9            boa   alta  adequada  acima_35  baixo
10           boa   alta   nenhuma      0_15   alto
12           boa   alta   nenhuma  acima_35  baixo
13          ruim   alta   nenhuma     15_35   alto

Total de registros: 11

X codificado:
[[2 0 1 0]
 [1 0 1 1]
 [1 1 1 2]
 [1 1 1 2]
 [1 1 0 2]
 [2 1 1 0]
 [0 1 1 2]
 [0 0 0 2]
 [0 0 1 0]
 [0 0 1 2]
 [2 0 1 1]]

y codificado:
[0 0 0 1 1 0 1 1 0 1 0]

Classes: ['alto' 'baixo']


## 3. Treinar o algoritmo de Regressão Logística

In [98]:
reglog_risco_credito = LogisticRegression(random_state=1)
reglog_risco_credito.fit(X_encoded, y_encoded)

print("Modelo de Regressão Logística treinado com sucesso!")

Modelo de Regressão Logística treinado com sucesso!


## 4. Verificar o intercepto (B0)

In [99]:
print(f"Intercepto (B0): {reglog_risco_credito.intercept_[0]}")

Intercepto (B0): -0.8085515186482205


## 5. Verificar os coeficientes

In [100]:
print(f"Coeficientes: {reglog_risco_credito.coef_}")

Coeficientes: [[-0.76706797  0.23921064 -0.47989768  1.12196145]]


## 6. Fazer predições com os exemplos

In [101]:
# Exemplo a) história boa, dívida alta, garantias nenhuma, renda > 35
exemplo_a = [
    label_encoders[0].transform(['boa'])[0],
    label_encoders[1].transform(['alta'])[0],
    label_encoders[2].transform(['nenhuma'])[0],
    label_encoders[3].transform(['acima_35'])[0]
]

# Exemplo b) história ruim, dívida alta, garantias adequada, renda < 15
exemplo_b = [
    label_encoders[0].transform(['ruim'])[0],
    label_encoders[1].transform(['alta'])[0],
    label_encoders[2].transform(['adequada'])[0],
    label_encoders[3].transform(['0_15'])[0]
]

pred_a = reglog_risco_credito.predict([exemplo_a])[0]
pred_b = reglog_risco_credito.predict([exemplo_b])[0]

classe_a = le_y.inverse_transform([pred_a])[0]
classe_b = le_y.inverse_transform([pred_b])[0]

print(f"\nPredição exemplo a) história boa, dívida alta, garantias nenhuma, renda > 35: {classe_a}")
print(f"Predição exemplo b) história ruim, dívida alta, garantias adequada, renda < 15: {classe_b}")


Predição exemplo a) história boa, dívida alta, garantias nenhuma, renda > 35: baixo
Predição exemplo b) história ruim, dívida alta, garantias adequada, renda < 15: alto


##Algoritmo de Regressão Logística para uma base de dados maior (Credit Data)

7. Agora aplique a Regressão Logística na base de dados ‘credit1.pkl’. De quanto foi a taxa de acerto?

8. O resultado com a base de dados ‘credit1.pkl’ é melhor que os resultados do Naive Bayes e das Florestas Aleatórias? Descreva sua análise de resultados (observe que para isso você deverá visualizar os resultados da Matriz de Confusão, acurácia, precisão e recall).

In [102]:
import pickle
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    classification_report,
)

with open('credit1.pkl', 'rb') as f:
    data = pickle.load(f)

if isinstance(data, (list, tuple)) and len(data) == 4:
    X_train, y_train, X_test, y_test = data
else:
    raise ValueError('arquivo invalido')

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

print('Shapes:')
print('X_train.shape:', X_train.shape)
print('y_train.shape:', y_train.shape)
print('X_test.shape:', X_test.shape)
print('y_test.shape:', y_test.shape)

logreg = LogisticRegression(random_state=1, solver='lbfgs', max_iter=2000)
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average='macro', zero_division=0)
rec = recall_score(y_test, y_pred, average='macro', zero_division=0)
cm = confusion_matrix(y_test, y_pred)

print('\nRESULTADOS DA REGRESSÃO LOGÍSTICA')
print('Acurácia:', acc)
print('Precisão (macro):', prec)
print('Revocação (macro):', rec)
print('\nMatriz de Confusão:\n', cm)
print('\nRelatório de Classificação:\n', classification_report(y_test, y_pred, zero_division=0))

Shapes:
X_train.shape: (1500, 3)
y_train.shape: (1500,)
X_test.shape: (500, 3)
y_test.shape: (500,)

RESULTADOS DA REGRESSÃO LOGÍSTICA
Acurácia: 0.946
Precisão (macro): 0.8808070901892412
Revocação (macro): 0.8757167431192661

Matriz de Confusão:
 [[423  13]
 [ 14  50]]

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.97      0.97      0.97       436
           1       0.79      0.78      0.79        64

    accuracy                           0.95       500
   macro avg       0.88      0.88      0.88       500
weighted avg       0.95      0.95      0.95       500



Não, a Regressão Logística não é melhor que o Naive Bayes nem que a Floresta Aleatória (Logistic acc 0.946 vs NB 0.982 e RF 0.97).
O Naive Bayes lidera (acc 0.982, recall classe1 0.95), RF é 2º (acc 0.97, recall 0.81) e Logistic tem recall bem menor para a classe minoritária (0.78), logo perde muitos positivos.

#PARTE 4: SVM

1. Aplique o algoritmo SVM na base de dados ‘credit.pkl’.
2. Inicialmente treine o SVM com kernel linear, valor do parâmetro C = 1.0 e ‘random_state =1’
3. Utilize o comando do sklearn accuray_score para calcular a acurácia do seu algoritmo. O resultado deve ser 0.946
4. Teste os demais kernels e anote os resultados. Qual o melhor kernel para a sua base de dados?
    * Polinomial
    * Sigmoide
    * rbf
5. Aumente o valor do parâmetro C aplicado ao melhor kernel e verifique se há mudanças no resultado do seu SVM.
6. O Grid Search (pesquisa em grade) é uma técnica utilizada para melhorar a precisão e a generalização dos modelos de aprendizado de máquina. Ela é usada para realizar ajustes de hiperparâmetros durante o treinamento de um modelo. O grid search automatiza o processo de encontrar hiperparâmetros ideais, economizando esforço humano em comparação com o ajuste manual, mas pode até ser mais custoso do ponto de vista de desempenho, pois testa todas as combinações possíveis e retorna a que obteve melhor desempenho.
Agora, aplique o GridSearch do Scikit-Learn (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) para ajudar a identificar os melhores hiperparâmetros para o seu modelo. Qual foi a melhor combinação de hiperparâmetros encontrada? O modelo com melhor desempenho foi obtido com os parâmetros ajustados manualmente ou com o GridSearch?

In [103]:
from sklearn.svm import SVC

In [104]:
import pickle
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

with open('credit1.pkl', 'rb') as f:
    data = pickle.load(f)

if isinstance(data, (list, tuple)) and len(data) == 4:
    X_train, y_train, X_test, y_test = data
else:
    raise ValueError('erro na leitura do arquivo')

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

results_svm = {}

print('\n' + '='*60)
print('1. SVM com kernel LINEAR')
print('='*60)
svc_linear = SVC(kernel='linear', C=1.0, random_state=1)
svc_linear.fit(X_train, y_train)
y_pred_lin = svc_linear.predict(X_test)
acc_lin = accuracy_score(y_test, y_pred_lin)
print(f'Acurácia: {acc_lin}')
print('Matriz de Confusão:\n', confusion_matrix(y_test, y_pred_lin))
print('\nRelatório de Classificação:\n', classification_report(y_test, y_pred_lin, zero_division=0))
results_svm['linear'] = acc_lin

# 2) Testar kernel POLINOMIAL
print('\n' + '='*60)
print('2. SVM com kernel POLINOMIAL')
print('='*60)
svc_poly = SVC(kernel='poly', C=1.0, random_state=1)
svc_poly.fit(X_train, y_train)
y_pred_poly = svc_poly.predict(X_test)
acc_poly = accuracy_score(y_test, y_pred_poly)
print(f'Acurácia: {acc_poly}')
print('Matriz de Confusão:\n', confusion_matrix(y_test, y_pred_poly))
print('\nRelatório de Classificação:\n', classification_report(y_test, y_pred_poly, zero_division=0))
results_svm['poly'] = acc_poly

# 3) Testar kernel SIGMOIDE
print('\n' + '='*60)
print('3. SVM com kernel SIGMOIDE')
print('='*60)
svc_sigmoid = SVC(kernel='sigmoid', C=1.0, random_state=1)
svc_sigmoid.fit(X_train, y_train)
y_pred_sigmoid = svc_sigmoid.predict(X_test)
acc_sigmoid = accuracy_score(y_test, y_pred_sigmoid)
print(f'Acurácia: {acc_sigmoid}')
print('Matriz de Confusão:\n', confusion_matrix(y_test, y_pred_sigmoid))
print('\nRelatório de Classificação:\n', classification_report(y_test, y_pred_sigmoid, zero_division=0))
results_svm['sigmoid'] = acc_sigmoid

# 4) Testar kernel RBF
print('\n' + '='*60)
print('4. SVM com kernel RBF')
print('='*60)
svc_rbf = SVC(kernel='rbf', C=1.0, random_state=1)
svc_rbf.fit(X_train, y_train)
y_pred_rbf = svc_rbf.predict(X_test)
acc_rbf = accuracy_score(y_test, y_pred_rbf)
print(f'Acurácia: {acc_rbf}')
print('Matriz de Confusão:\n', confusion_matrix(y_test, y_pred_rbf))
print('\nRelatório de Classificação:\n', classification_report(y_test, y_pred_rbf, zero_division=0))
results_svm['rbf'] = acc_rbf

# Comparação e identificação do melhor kernel
print('\n' + '='*60)
print('COMPARAÇÃO DE KERNELS')
print('='*60)
for kernel, acc in results_svm.items():
    print(f'{kernel:12s}: {acc:.4f}')

best_kernel = max(results_svm.items(), key=lambda kv: kv[1])
print(f'\nMelhor kernel: {best_kernel[0]} com acurácia = {best_kernel[1]:.4f}')



1. SVM com kernel LINEAR
Acurácia: 0.946
Matriz de Confusão:
 [[422  14]
 [ 13  51]]

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.97      0.97      0.97       436
           1       0.78      0.80      0.79        64

    accuracy                           0.95       500
   macro avg       0.88      0.88      0.88       500
weighted avg       0.95      0.95      0.95       500


2. SVM com kernel POLINOMIAL
Acurácia: 0.968
Matriz de Confusão:
 [[433   3]
 [ 13  51]]

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.97      0.99      0.98       436
           1       0.94      0.80      0.86        64

    accuracy                           0.97       500
   macro avg       0.96      0.89      0.92       500
weighted avg       0.97      0.97      0.97       500


3. SVM com kernel SIGMOIDE
Acurácia: 0.838
Matriz de Confusão:
 [[393  43]
 [ 38  26]]

Relatório de Classifi

Aumentando o C do kernel RBF para 2, teve um leve aumento na acuracia, totalizando 0.988. aumentando para 5, o resultado vai para 0.986, indicando que tem um limite máximo para melhoria do valor C






# Análise dos resultados dos 4 algoritmos utilizados:

6. O resultado do SVM é melhor que os resultados do Naive Bayes, Florestas Aleatórias e Regressão Logística? Descreva sua análise de resultados (observe que para isso você deverá visualizar os resultados da Matriz de Confusão, acurácia, precisão e recall).
O SVM com RBF apresentou maior acurácia (0.982) e melhores métricas globais.

A matriz mostra quase nenhum falso positivo/negativo e um recall alto para a classe 0 e aceitável para a classe 1.
Comparando com NB, RF e LR, o SVM generalizou melhor e manteve maior equilíbrio entre precisão e recall.

In [105]:
import pickle, numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

with open('credit1.pkl', 'rb') as f:
    X_train, y_train, X_test, y_test = pickle.load(f)

X_train, X_test = np.asarray(X_train), np.asarray(X_test)

param_grid = {
    'C': [0.3, 1, 3, 10, 30, 100],
    'gamma': ['scale', 'auto', 0.1, 0.03, 0.01, 0.003, 0.001],
    'kernel': ['rbf']
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = {
    'accuracy': 'accuracy',
    'f1_macro': 'f1_macro',
    'recall_macro': 'recall_macro',
    'precision_macro': 'precision_macro'
}

grid = GridSearchCV(
    SVC(class_weight=None), # possivel ajustar o peso
    param_grid=param_grid,
    scoring=scoring,
    refit='f1_macro',
    cv=cv,
    n_jobs=-1,
    verbose=0,
    return_train_score=False
)

grid.fit(X_train, y_train)

print("===== MELHOR COMBINAÇÃO (CV pelo f1_macro) =====")
print(grid.best_params_)
print(f"Melhor f1_macro (CV): {grid.best_score_:.4f}")

best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
cm  = confusion_matrix(y_test, y_pred)
print("\n===== DESEMPENHO NO TESTE =====")
print(f"Acurácia (teste): {acc:.3f}")
print("Matriz de Confusão (teste):\n", cm)
print("\nRelatório de Classificação (teste):")
print(classification_report(y_test, y_pred, digits=3))


===== MELHOR COMBINAÇÃO (CV pelo f1_macro) =====
{'C': 100, 'gamma': 'scale', 'kernel': 'rbf'}
Melhor f1_macro (CV): 0.9839

===== DESEMPENHO NO TESTE =====
Acurácia (teste): 0.990
Matriz de Confusão (teste):
 [[435   1]
 [  4  60]]

Relatório de Classificação (teste):
              precision    recall  f1-score   support

           0      0.991     0.998     0.994       436
           1      0.984     0.938     0.960        64

    accuracy                          0.990       500
   macro avg      0.987     0.968     0.977       500
weighted avg      0.990     0.990     0.990       500



# Conclusão
# Descreva aqui suas conclusões sobre os resultados obtidos com os diferentes algoritmos e a importância da escolha adequada do modelo e hiperparâmetros.
Os resultados mostraram que modelos diferentes apresentam desempenhos distintos, mesmo usando a mesma base.
Entre os algoritmos testados, o SVM com kernel RBF foi o que obteve o melhor desempenho geral.
Além disso, o GridSearch ainda conseguiu melhorar esse SVM, aumentando acurácia e F1, principalmente para a classe minoritária.
Desta forma, a escolha correta dos hiperparametros impacta direto o resultado final. Ou seja, é necessário testar diferentes combinações para encontrar a melhor configuração, e obter a melhor performance depende da habilidade da pessoa que está desenvolvendo o modelo.